In [273]:
import MetaTrader5 as mt5 #mt5 to access historcal data
import pandas as pd # for data analysis and calculation of technical indcator
import numpy as np
import plotly.io as pio
import plotly.express as px # for data visualization
import plotly.graph_objects as go
import tdclient as TDClient
from datetime import datetime, time ,timezone #to specify the date time range for historical data
from IPython.display import display, Markdown, Latex # to display result in python notebook
from backtest import Backtester, get_ohlc_history, create_price_fig, evaluate_backtest

In [274]:
# Conecte mt5
mt5.initialize()
# logine mt5
login = 52185665
password = '3cAJz$AQiKwMq0'
server = 'ICMarketsSC-Demo'

mt5.login(login,password,server)

True

In [275]:
# this function retreives olhc data from mt5 account and return a data frame
def get_ohlc(symbol, timeframe, start_datetime, end_datetime):
    ohlc = mt5.copy_rates_range(symbol, timeframe, start_datetime, end_datetime)
    ohlc_df = pd.DataFrame(ohlc)
    ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit= 's')
    return ohlc_df[['time', 'open', 'high', 'low', 'close']]


In [ ]:
# 30 minut df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_M1
start_datetime = datetime(2025,3,15)
end_datetime = datetime.now()
ohlc_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
ohlc_df

In [277]:
# hour4 df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_H4
hourly_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
hourly_df.tail(5)

,time,open,high,low,close
235,2025-05-09 04:00:00,1.12271,1.12300,1.11968,1.12279
236,2025-05-09 08:00:00,1.12279,1.12593,1.12185,1.12565
237,2025-05-09 12:00:00,1.12565,1.12606,1.12425,1.12541
238,2025-05-09 16:00:00,1.12541,1.12929,1.12532,1.12586
239,2025-05-09 20:00:00,1.12586,1.12664,1.12473,1.12497


In [278]:
# daily df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_D1
daily_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
# Convert 'time' column to full timestamp
#daily_df['time'] = pd.to_datetime(daily_df['time']).dt.strftime('%Y-%m-%d %H:%M:%S')
#daily_df['time'] = pd.to_datetime(daily_df['time'], unit= 's').dt.strftime('%Y-%m-%d %H:%M:%S')
daily_df

,time,open,high,low,close
0,2025-03-17,1.08771,1.09297,1.08686,1.09214
1,2025-03-18,1.09218,1.09547,1.08925,1.09447
2,2025-03-19,1.09439,1.09461,1.08606,1.09025
3,2025-03-20,1.09010,1.09176,1.08147,1.08533
4,2025-03-21,1.08494,1.08613,1.07973,1.08124
5,2025-03-24,1.08127,1.08584,1.07817,1.08008
6,2025-03-25,1.08003,1.08302,1.07771,1.07913
7,2025-03-26,1.07901,1.08028,1.07435,1.07525
8,2025-03-27,1.07539,1.08212,1.07330,1.08018
9,2025-03-28,1.08011,1.08448,1.07645,1.08306


In [279]:
# Calculate Pivot Point, R1, R2, R3, stopeR4, S1, S2, S3, stopeS4 levels with Fibonacci 
daily_df["pivot"] = ((daily_df["high"] + daily_df["low"] + daily_df["close"]) / 3).shift(1)
daily_df["prange"] = (daily_df["high"] - daily_df["low"]).shift(1)
daily_df["R1"] = daily_df["pivot"] + 0.382 * daily_df["prange"]
daily_df["R2"] = daily_df["pivot"] + 0.618 * daily_df["prange"]
daily_df["R3"] = daily_df["pivot"] + 1.000 * daily_df["prange"]

daily_df["S1"] = daily_df["pivot"] - 0.382 * daily_df["prange"]
daily_df["S2"] = daily_df["pivot"] - 0.618 * daily_df["prange"]
daily_df["S3"] = daily_df["pivot"] - 1.000 * daily_df["prange"]
daily_df[["time","S2","R2"]].tail(10)


,time,S2,R2
30,2025-04-28,1.130904,1.140223
31,2025-04-29,1.133295,1.145099
32,2025-04-30,1.135971,1.142349
33,2025-05-01,1.129784,1.139956
34,2025-05-02,1.125253,1.134547
35,2025-05-05,1.125229,1.138417
36,2025-05-06,1.127957,1.136856
37,2025-05-07,1.128069,1.140577
38,2025-05-08,1.127018,1.137709
39,2025-05-09,1.118189,1.133565


In [280]:
#setting the sma 1 and 14 periods
furst_sma_period = 1
second_sma_period = 14
#ohlc4
hourly_df['ohlc4'] = (hourly_df['open'] + hourly_df['high'] + hourly_df['low'] + hourly_df['close']) / 4

# sma 1
hourly_df['sma_1'] = hourly_df['ohlc4'].rolling(furst_sma_period).mean()
hourly_df['prev_sma_1'] = hourly_df['sma_1'].shift(1) #to find crossovers, previous sma value is necessary using shift()
# sma 14
hourly_df['sma_14'] = hourly_df['ohlc4'].rolling(second_sma_period).mean()
hourly_df

,time,open,high,low,close,ohlc4,sma_1,prev_sma_1,sma_14
0,2025-03-17 00:00:00,1.08771,1.08855,1.08706,1.08795,1.087818,1.087818,NaN,NaN
1,2025-03-17 04:00:00,1.08795,1.08940,1.08742,1.08822,1.088247,1.088247,1.087818,NaN
2,2025-03-17 08:00:00,1.08822,1.08869,1.08686,1.08838,1.088037,1.088037,1.088247,NaN
3,2025-03-17 12:00:00,1.08838,1.09119,1.08771,1.09018,1.089365,1.089365,1.088037,NaN
4,2025-03-17 16:00:00,1.09018,1.09297,1.08972,1.09212,1.091248,1.091248,1.089365,NaN
...,...,...,...,...,...,...,...,...,...
235,2025-05-09 04:00:00,1.12271,1.12300,1.11968,1.12279,1.122045,1.122045,1.122428,1.130309
236,2025-05-09 08:00:00,1.12279,1.12593,1.12185,1.12565,1.124055,1.124055,1.122045,1.129509
237,2025-05-09 12:00:00,1.12565,1.12606,1.12425,1.12541,1.125342,1.125342,1.124055,1.128886
238,2025-05-09 16:00:00,1.12541,1.12929,1.12532,1.12586,1.126470,1.126470,1.125342,1.128235


In [281]:
#setting the 11 and 114 sma periods
thurd_sma_period = 1
fourth_sma_period = 14
#ohlc4
daily_df['ohlc4'] = (daily_df['open'] + daily_df['high'] + daily_df['low'] + daily_df['close']) / 4
# sma 11
daily_df['sma_11'] = daily_df['ohlc4'].rolling(thurd_sma_period).mean()
daily_df['prev_sma_11'] = daily_df['sma_11'].shift(1) #to find crossovers, previous sma value is necessary using shift()
# sma 114
daily_df['sma_114'] = daily_df['ohlc4'].rolling(fourth_sma_period).mean()
daily_df[["time","sma_11","sma_114","prev_sma_11"]]

,time,sma_11,sma_114,prev_sma_11
0,2025-03-17,1.089920,NaN,NaN
1,2025-03-18,1.092842,NaN,1.089920
2,2025-03-19,1.091328,NaN,1.092842
3,2025-03-20,1.087165,NaN,1.091328
4,2025-03-21,1.083010,NaN,1.087165
5,2025-03-24,1.081340,NaN,1.083010
6,2025-03-25,1.079973,NaN,1.081340
7,2025-03-26,1.077222,NaN,1.079973
8,2025-03-27,1.077748,NaN,1.077222
9,2025-03-28,1.081025,NaN,1.077748


In [282]:
#setting entring and closing time and one trade a day
# Set the start and end times for your desired trading hours
start_time = datetime.strptime('06:00:00', '%H:%M:%S').time()
end_time = datetime.strptime('17:00:00', '%H:%M:%S').time()
end_trading_section = datetime.strptime('23:45:00', '%H:%M:%S').time()

# Filter the trade data for trades within the specified trading hours
#trades_within_hours = daily_df[(daily_df['time'].dt.time >= start_time) & (daily_df['time'].dt.time <= end_time)]
# open one trade a day
first_trade_per_day = daily_df.groupby(daily_df['time'].dt.time).first()
# Close all open positions at 11:45 PM
open_positions = daily_df[daily_df['time'].dt.time == end_trading_section]

# function to trading
daily_df["sell_stope_loss"] = (daily_df["R2"] - daily_df["S1"]) /2 + daily_df["R2"] 
daily_df["buy_stope_loss"] = (daily_df["S2"] - ((daily_df["R1"] - daily_df["S2"]) /2))
daily_df['sell_profit'] = daily_df['low'] <= daily_df["S1"]
daily_df['buy_profit'] = daily_df["high"] >= daily_df["R1"]
daily_df

,time,open,high,low,close,pivot,prange,R1,R2,R3,...,S2,S3,ohlc4,sma_11,prev_sma_11,sma_114,sell_stope_loss,buy_stope_loss,sell_profit,buy_profit
0,2025-03-17,1.08771,1.09297,1.08686,1.09214,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.089920,1.089920,NaN,NaN,NaN,NaN,False,False
1,2025-03-18,1.09218,1.09547,1.08925,1.09447,1.090657,0.00611,1.092991,1.094433,1.096767,...,1.086881,1.084547,1.092842,1.092842,1.089920,NaN,1.097488,1.083826,False,True
2,2025-03-19,1.09439,1.09461,1.08606,1.09025,1.093063,0.00622,1.095439,1.096907,1.099283,...,1.089219,1.086843,1.091328,1.091328,1.092842,NaN,1.100017,1.086109,True,False
3,2025-03-20,1.09010,1.09176,1.08147,1.08533,1.090307,0.00855,1.093573,1.095591,1.098857,...,1.085023,1.081757,1.087165,1.087165,1.091328,NaN,1.099866,1.080748,True,False
4,2025-03-21,1.08494,1.08613,1.07973,1.08124,1.086187,0.01029,1.090117,1.092546,1.096477,...,1.079827,1.075897,1.083010,1.083010,1.087165,NaN,1.097691,1.074682,True,False
5,2025-03-24,1.08127,1.08584,1.07817,1.08008,1.082367,0.00640,1.084811,1.086322,1.088767,...,1.078411,1.075967,1.081340,1.081340,1.083010,NaN,1.089522,1.075211,True,True
6,2025-03-25,1.08003,1.08302,1.07771,1.07913,1.081363,0.00767,1.084293,1.086103,1.089033,...,1.076623,1.073693,1.079973,1.079973,1.081340,NaN,1.089938,1.072788,True,False
7,2025-03-26,1.07901,1.08028,1.07435,1.07525,1.079953,0.00531,1.081982,1.083235,1.085263,...,1.076672,1.074643,1.077222,1.077222,1.079973,NaN,1.085890,1.074017,True,False
8,2025-03-27,1.07539,1.08212,1.07330,1.08018,1.076627,0.00593,1.078892,1.080291,1.082557,...,1.072962,1.070697,1.077748,1.077748,1.077222,NaN,1.083256,1.069997,True,True
9,2025-03-28,1.08011,1.08448,1.07645,1.08306,1.078533,0.00882,1.081903,1.083984,1.087353,...,1.073083,1.069713,1.081025,1.081025,1.077748,NaN,1.088394,1.068673,False,True


In [283]:
ohlc_df = ohlc_df.sort_values('time')
daily_df = daily_df.sort_values('time')
def entring_stoploss_profit(row, ohlc_df) :
    if row["prev_sma_11"] > row['sma_114'] and row["low_x"] <= row["S2"] :
        return "buy"
    elif row['prev_sma_11'] < row['sma_114'] and row["high_x"] >= row["R2"]:
        return "sell"
    # Merge daily indicators into minute data (asof = merge on closest earlier date)
merged_df = pd.merge_asof(ohlc_df, daily_df, on='time', direction='backward')
merged_df["signal"] = merged_df.apply(entring_stoploss_profit, axis=1, ohlc_df = ohlc_df)

# Tag first signal per day in original merged_df
merged_df['is_first_signal'] = (merged_df[merged_df["signal"].notna()].groupby(merged_df["time"].dt.date).cumcount() == 0)

# Set signal only on the first row per day, others = None
merged_df['signal'] = merged_df.apply(lambda row: row['signal'] if row['is_first_signal'] else None, axis=1)
merged_df[['signal', 'time']].dropna()

,signal,time
19208,sell,2025-04-03 08:28:00
21585,buy,2025-04-07 00:09:00
31365,buy,2025-04-15 19:17:00
38872,buy,2025-04-23 00:35:00
46324,buy,2025-04-30 04:57:00
49664,sell,2025-05-02 12:39:00
52817,sell,2025-05-06 17:16:00


In [284]:
merged_df = merged_df.rename(columns={
    "low_x": "low",
    "high_x": "high",
    "open_x": "open",
    "close_x": "close",
    "low_y": "daily_low",
    "high_y": "daily_high",
    "open_y": "daily_open",
    "close_y": "daily_close"
})


In [285]:
print(merged_df.columns.tolist())



['time', 'open', 'high', 'low', 'close', 'daily_open', 'daily_high', 'daily_low', 'daily_close', 'pivot', 'prange', 'R1', 'R2', 'R3', 'S1', 'S2', 'S3', 'ohlc4', 'sma_11', 'prev_sma_11', 'sma_114', 'sell_stope_loss', 'buy_stope_loss', 'sell_profit', 'buy_profit', 'signal', 'is_first_signal']


In [286]:
# create trade logic
def on_bar(data, trades, orders):
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    if 'signal' not in data:
        print("Warning: 'signal' column is missing!")
        return
    account_balance = 10000  # your account in USD
    risk_percent = 0.20  # 2%
    pip_value_per_lot = 10  # on EURUSD, 1 lot = $10 per pip
    risk_amount = account_balance * risk_percent
    # entry signal
    if data['signal'] == 'buy' and not num_open_trades:
        stop_loss_pips = data['S2'] - (data["S2"] - ((data["R1"] - data["S2"]) /2)) # example SL
        volume = risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'buy')
    
    elif data['signal'] == 'sell' and not num_open_trades:
        stop_loss_pips = ((data["R2"] - data["S1"]) /2 + data["R2"]) - data["R2"]
        volume = risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'sell')
        
# exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if trade['order_type'] == 'buy' and ((data["low"] <= data["S2"] - ((data["R1"] - data["S2"]) /2)) 
                                            or (data["high"] >= data["R1"]) 
                                            or (data['time'].time() == pd.to_datetime('00:00:00').time())):
            orders.close_trade(trade)
        elif trade['order_type'] == 'sell' and ((data["high"] >= (data["R2"] - data["S1"]) /2 + data["R2"]) 
                                            or (data["low"] <= data["S1"]) 
                                            or (data['time'].time() == pd.to_datetime('00:00:00').time())):
            orders.close_trade(trade)

In [287]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1
commission = -7 / 100000

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(merged_df)
bt.set_on_bar(on_bar)

bt.run_backtest()

bt.trades.tail(7)

,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,EURUSD,sell,27797.081306,2025-04-03 08:28:00,1.09366,2025-04-03 11:40:00,1.10087,0,0,{},-200.42,-1.945796,-202.365796,-202.365796,9797.634204
1,closed,EURUSD,buy,21834.061135,2025-04-07 00:09:00,1.08857,2025-04-08 00:00:00,1.09066,0,0,{},45.63,-1.528384,44.101616,-158.26418,9841.73582
2,closed,EURUSD,buy,31104.199067,2025-04-15 19:17:00,1.12794,2025-04-16 00:00:00,1.12781,0,0,{},-4.04,-2.177294,-6.217294,-164.481474,9835.518526
3,closed,EURUSD,buy,30792.917629,2025-04-23 00:35:00,1.13826,2025-04-23 01:29:00,1.13176,0,0,{},-200.15,-2.155504,-202.305504,-366.786978,9633.213022
4,closed,EURUSD,buy,77519.379845,2025-04-30 04:57:00,1.13631,2025-04-30 18:29:00,1.1337,0,0,{},-202.33,-5.426357,-207.756357,-574.543335,9425.456665
5,closed,EURUSD,sell,53191.489362,2025-05-02 12:39:00,1.13425,2025-05-05 00:00:00,1.13037,0,0,{},206.38,-3.723404,202.656596,-371.886739,9628.113261
6,closed,EURUSD,sell,55555.555556,2025-05-06 17:16:00,1.13593,2025-05-07 00:00:00,1.13691,0,0,{},-54.44,-3.888889,-58.328889,-430.215628,9569.784372


In [288]:
mm = bt.plot_balance()
display(mm)

In [289]:
pnl_chart = bt.plot_pnl()
display(pnl_chart)

In [290]:
evaluate_backtest(bt.trades)

biggest_profit 206.38
daily_drawdown -202.33
max_drawown -416.2791547578547
avg_win 126.005
avg_loss -132.276
count_profit_trades 2
count_loss_trades 5
Win Rate1: 28.57%
win_rate 0.4
rrr 0.95


,order_type,profit
0,buy,-360.89
1,sell,-48.48


AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
# visualizing the ohlc data
fig = go.Figure(data = [go.Candlestick(x = ohlc_df['time'],
                                        open = ohlc_df['open'],
                                        high = ohlc_df['high'],
                                        low = ohlc_df['low'],
                                        close = ohlc_df['close'])])

# fig.update_layout(height=600, xaxis_rangeslider_visible=False)
# 📌 Add Pivot Points as Horizontal Lines
pivot_levels = ["sell_stope_loss", "R2", "buy_stope_loss", "S2"]
colors = {'sell_stope_loss': 'red', 'buy_stope_loss': 'red', 'R2': 'green', 'S2': 'green'}


for _, row in daily_df.iterrows():
    for level in pivot_levels:
        fig.add_trace(go.Scatter(
            x=[row['time'], row['time'] + pd.Timedelta(days=1)],  # Extend for one day
            y=[row[level], row[level]],
            mode='lines',
            name=f"{level} {row['time']}",
            line=dict(color=colors[level], width=1.5, dash='dot')
        ))

# 📌 Customize Layout
fig.update_layout(
    title="Pivot Points on Candlestick Chart",
    xaxis_title="Date",
    yaxis_title="Price",
    template="plotly_dark",
    xaxis_rangeslider_visible=False
)
fig